In [1]:
import numpy as np
import math
import pandas as pd

np.seterr(all="raise")  # Error on overflow

# Constants
LOAD_DEAD = 8000  # kN
LOAD_LIVE = 2500  # kN
BEARINGPRESSURE_ALLOWABLE = 150  # kPa
FTG_COVER = 0.060  # m
COLUMN_WIDTH = 0.500  # m


class FoundationSizes:
    def __init__(self):
        """Initialize the FoundationSizes class and calculate all properties."""
        self.sizes = self.generate_foundation_sizes()
        self.initialize_attributes()
        self.calculate_all_properties()

    def generate_foundation_sizes(self):
        """Generate the foundation sizes based on initial parameters."""
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt((LOAD_DEAD + LOAD_LIVE) / BEARINGPRESSURE_ALLOWABLE) / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * COLUMN_WIDTH**2) / 3
                        + (1 + (3 * BEARINGPRESSURE_ALLOWABLE) / 19040)
                        * (LOAD_DEAD + LOAD_LIVE)
                    )
                    - (9520 + 3 * BEARINGPRESSURE_ALLOWABLE) * COLUMN_WIDTH
                )
                / (19040 + 3 * BEARINGPRESSURE_ALLOWABLE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.01, 0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        # Instantiate the full array with additional empty columns
        sizes = np.zeros((foundation_sizes.shape[0], 27), dtype=np.float32)
        sizes[:, :5] = foundation_sizes
        return sizes

    def initialize_attributes(self):
        """Initialize attributes for foundation properties."""
        attributes = [
            "FtgLength",
            "FtgDepth",
            "fc",
            "ReoSize",
            "ReoCts",
            "BPmax",
            "BP_Ratio",
            "Pult",
            "Dom",
            "BPult",
            "CLR",
            "VPult",
            "fVP",
            "VP_Ratio",
            "dv",
            "VLult",
            "kvo",
            "kv",
            "ks",
            "fVuc",
            "VL_Ratio",
            "Mult",
            "Astshr",
        ]

        for i, attr in enumerate(attributes):
            setattr(self, attr, self.sizes[:, i])

    def calculate_all_properties(self):
        """Calculate all foundation properties."""
        calc_methods = [
            self.calc_BPmax,
            self.calc_Pult,
            self.calc_ratio_bearing,
            self.calc_Dom,
            self.calc_BPult,
            self.calc_CLR,
            self.calc_VPult,
            self.calc_fVP,
            self.calc_VP_Ratio,
            self.calc_dv,
            self.calc_VLult,
            self.calc_kvo,
            self.calc_kv,
            self.calc_ks,
            self.calc_fVuc,
            self.calc_VL_Ratio,
            self.calc_Mult,
            self.calc_Astshr,
        ]

        for method in calc_methods:
            method()

    def calc_BPmax(self):
        """Calculate maximum bearing pressure."""
        self.BPmax[:] = (
            6 * self.FtgDepth * self.FtgLength**2 + LOAD_LIVE + LOAD_DEAD
        ) / (self.FtgLength**2)

    def calc_ratio_bearing(self):
        """Calculate bearing pressure ratio."""
        self.BP_Ratio[:] = BEARINGPRESSURE_ALLOWABLE / self.BPmax

    def calc_Pult(self):
        """Calculate ultimate load."""
        self.Pult[:] = (
            1.2 * (6 * self.FtgDepth * self.FtgLength**2 + LOAD_DEAD) + 1.5 * LOAD_LIVE
        )

    def calc_Dom(self):
        """Calculate depth of the moment arm."""
        self.Dom[:] = self.FtgDepth - FTG_COVER - self.ReoSize / 2

    def calc_BPult(self):
        """Calculate ultimate bearing pressure."""
        self.BPult[:] = self.Pult / self.FtgLength**2

    def calc_CLR(self):
        """Calculate column load ratio."""
        self.CLR[:] = self.BPult * (COLUMN_WIDTH + self.Dom) ** 2

    def calc_VPult(self):
        """Calculate ultimate vertical load."""
        self.VPult[:] = self.Pult - self.CLR

    def calc_fVP(self):
        """Calculate vertical load capacity"""
        self.fVP[:] = (
            952
            * (self.ReoSize - self.Dom)
            * (self.ReoSize - self.Dom - COLUMN_WIDTH)
            * np.sqrt(self.fc)
        )

    def calc_VP_Ratio(self):
        """Calculate vertical load capacity ratio"""
        self.VP_Ratio[:] = self.VPult / self.fVP

    def calc_dv(self):
        """Calculate vertical load displacement"""
        self.dv[:] = np.maximum(0.9 * self.Dom, 0.72 * self.FtgDepth)

    def calc_VLult(self):
        """Calculate ultimate vertical load"""
        self.VLult[:] = (
            -0.5 * self.BPult * (COLUMN_WIDTH + 2 * self.dv - self.FtgLength)
        )

    def calc_kvo(self):
        self.kvo[:] = 2 / (10 + 13 * self.dv)

    def calc_kv(self):
        self.kv[:] = np.minimum(self.kvo, 0.15)

    def calc_ks(self):
        self.ks[:] = np.maximum(0.5, (10 / 7) * (1 - self.FtgDepth))

    def calc_fVuc(self):
        self.fVuc[:] = (
            700 * self.FtgDepth * self.dv * np.sqrt(self.fc) * self.ks * self.kv
        )

    def calc_VL_Ratio(self):
        self.VL_Ratio[:] = self.VLult / self.fVuc

    def calc_Mult(self):
        self.Mult[:] = (
            self.BPult
            / 800
            * (7 * COLUMN_WIDTH - 10 * self.FtgLength) ** 2
            * self.FtgLength
        )

    def calc_Astshr(self):
        self.Astshr[:] = (5 * self.Mult) / (2 * self.Dom * self.FtgLength)

    def print_raw_array(self, num_rows=5):
        """Print the raw array of foundation sizes."""
        print(self.sizes[0, :])

    def print_foundation_sizes(self, num_rows=6):
        """Print a sample of the foundation sizes."""
        column_headers = [
            attr
            for attr in self.__dict__
            if isinstance(getattr(self, attr), np.ndarray)
        ][1:]
        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        num_columns = len(column_headers)

        total_rows = self.sizes.shape[0]
        step = max(1, total_rows // (num_rows - 1))

        selected_rows = np.arange(0, total_rows, step)
        if len(selected_rows) < num_rows:
            selected_rows = np.append(selected_rows, total_rows - 1)
        elif len(selected_rows) > num_rows:
            selected_rows = selected_rows[:num_rows]

        df = pd.DataFrame(
            self.sizes[selected_rows, :num_columns],
            columns=column_headers[:num_columns],
            index=selected_rows,
        )
        print(df.to_string(header=True))


# Create an instance of the FoundationSizes class
foundation = FoundationSizes()
foundation.print_foundation_sizes()
foundation.sizes.shape


        FtgLength  FtgDepth     fc  ReoSize  ReoCts   BPmax  BP_Ratio      Pult   Dom   BPult     CLR     VPult       fVP  VP_Ratio    dv   VLult   kvo    kv    ks    fVuc  VL_Ratio      Mult   Astshr
0           8.400     1.050 20.000    0.010   0.080 155.110     0.967 13883.435 0.985 196.761 433.902 13449.533  6122.778     2.197 0.886 602.776 0.093 0.093 0.500 135.378     4.453 13388.115 4045.237
89812       8.800     1.450 32.000    0.028   0.125 144.289     1.040 14158.475 1.376 182.832 643.453 13515.021 13415.405     1.007 1.238 532.332 0.077 0.077 0.500 272.442     1.954 14360.092 2964.805
179624      9.200     1.850 50.000    0.012   0.175 135.155     1.110 14477.405 1.784 171.047 892.292 13585.113 27101.543     0.501 1.606 469.421 0.065 0.065 0.500 476.229     0.986 15406.339 2346.696
269436      8.600     1.150 25.000    0.036   0.220 148.869     1.008 13962.390 1.072 188.783 466.517 13495.872  7574.569     1.782 0.965 582.433 0.089 0.089 0.500 172.267     3.381 13812.720 3745

(449064, 27)